In [9]:
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
import numpy as np
import os
import datetime
import pandas as pd
import xarray as xr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from PIL import Image

root_dir='/glade/u/home/hongli/work/sharp/basins/scripts/'
os.chdir(root_dir)
output_folder='step3_plot_sim_hydro'
outputfile = 'hydrgraph.png'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

nldas_sim_file='/glade/u/home/hongli/work/sharp/basins/tayprk_huc12_nldas/route/output/all.nc'
gmet_sim_file='/glade/u/home/hongli/work/sharp/basins/tayprk_huc12_metsim/route/output/all.nc'
obs_file='Taylor_park_09107000.txt'
q_vname2 = 'IRFroutedRunoff'
t_vname = 'time'
time_format='%Y-%m-%d'

plot_start_date_summary=['2010-01-02']
plot_end_date_summary = ['2010-12-31']
formatter = DateFormatter('%m/%d/%y')
# formatter = DateFormatter('%b-%y')
dpi_value= 100

# read
print('read data')

# read NLDAS simulated flow (m^3/s)
f = xr.open_dataset(nldas_sim_file)
nldas_sim_irf = f[q_vname2].values[:]
nldas_sim_time = f[t_vname].dt.strftime(time_format).values[:]
f.close()

sim_start_str=nldas_sim_time[0]
sim_end_str=nldas_sim_time[-1]

# read GMET simulated flow (m^3/s)
f = xr.open_dataset(gmet_sim_file)
gmet_sim_irf = f[q_vname2].values[:]
gmet_sim_time = f[t_vname].dt.strftime(time_format).values[:]
f.close()
gmet_start=(datetime.datetime.strptime(sim_start_str,time_format) - datetime.datetime.strptime(gmet_sim_time[0],time_format)).days
gmet_len=(datetime.datetime.strptime(sim_end_str,time_format) - datetime.datetime.strptime(sim_start_str,time_format)).days+1
gmet_sim_irf_useful = gmet_sim_irf[gmet_start:gmet_start+gmet_len]
  
# read observed flow (f^3/s)
obs = []
obs_time = []
with open(os.path.join(root_dir, obs_file)) as f:
    for line in f:
        line = line.strip()
        if line and line.startswith('USGS'):
            splits = line.split()
            obs_time.append(splits[2])
            if len(splits)>3:#flow data exists
                obs.append(float(splits[3])/35.315) #convert to m^3/s
            else:
                obs.append(np.nan)
obs_start=(datetime.datetime.strptime(sim_start_str,time_format) - datetime.datetime.strptime(obs_time[0],time_format)).days
obs_len=(datetime.datetime.strptime(sim_end_str,time_format) - datetime.datetime.strptime(sim_start_str,time_format)).days+1
obs_useful = obs[obs_start:obs_start+obs_len]
         
# plot
print('plot')
os.chdir(os.path.join(root_dir,output_folder))
fig_files = []
for m in range(len(plot_start_date_summary)):

    fig_file='image_'+str(m+1)+'.png'
    fig_files.append(fig_file)
    
    row_num=1
    col_num=1        
    fig, ax = plt.subplots(row_num,col_num)
    fig.set_figwidth(6.5*col_num) #190mm
    fig.set_figheight(6.5*0.5*row_num) #5.61 heigh/width=3/4

    plot_start_str=plot_start_date_summary[m]
    plot_end_str=plot_end_date_summary[m]

    plot_start=((datetime.datetime.strptime(plot_start_str,time_format)-datetime.datetime.strptime(sim_start_str,time_format)).days)
    plot_len=((datetime.datetime.strptime(plot_end_str,time_format)-datetime.datetime.strptime(plot_start_str,time_format)).days+1)

    plot_start_date=datetime.datetime.strptime(plot_start_str,time_format)
    t=[plot_start_date+datetime.timedelta(days=d) for d in range(0,plot_len)]
    t_plot=[mpl.dates.date2num(d) for d in t[0:plot_len]]        

    obs_plot=obs_useful[plot_start+0:plot_start+plot_len]
    nldas_sim_irf_plot=nldas_sim_irf[plot_start+0:plot_start+plot_len]
    gmet_sim_irf_plot=gmet_sim_irf_useful[plot_start+0:plot_start+plot_len]

    ax.plot_date(t_plot, gmet_sim_irf_plot, 'b-o', linewidth=1, markersize=1.0, alpha=0.8, label='GMET simulation') 
    ax.plot_date(t_plot, nldas_sim_irf_plot, 'r-^', linewidth=1, markersize=1.0, alpha=0.8, label='NLDAS simulation') 
    ax.plot_date(t_plot, obs_plot, 'k-s', linewidth=1.0, markersize=1.0, alpha=0.8, label='Observation')   

    #axis, label, title, legend
#         ax.set_title(titles[i], fontsize='small', weight='semibold')
    ax.set_xlabel('Date', fontsize='small')
    ax.set_ylabel('Flow $(m^3/s)$', fontsize='small')
    ax.xaxis.set_major_formatter(formatter)
    ax.xaxis.set_tick_params(labelsize='small')#rotation=30,
    ax.legend(loc='best', fontsize='x-small', framealpha=0.5) #loc='upper right', 

    plt.rc('xtick',labelsize='small')
    plt.rc('ytick',labelsize='small')                
    fig.tight_layout()
    fig.savefig(fig_file, dpi=dpi_value)
    plt.close(fig)      
    del fig

# save as one figure
print('save')
widths = []
heights = []
for fig_file in fig_files:
    im = Image.open(fig_file)
    widths.append(im.width)
    heights.append(im.height)

max_width = max(widths)
total_height = sum(heights)
new_im = Image.new('RGB', (max_width, total_height))

x_offset = 0
for fig_file in fig_files:
    im = Image.open(fig_file)    
    new_im.paste(im, (0,x_offset))
    x_offset += im.size[1]
new_im.save(outputfile)

for file in os.listdir(os.getcwd()):
    if file.startswith('image'):
        os.remove(file)
[os.remove(file) for file in os.getcwd() if file.startswith('image')]   

os.chdir(root_dir)


read data
plot
save


In [3]:
gmet_sim_time[0]

numpy.datetime64('2008-01-02T00:00:00.000000000')

In [4]:
datetime.datetime.strptime(sim_start_str,time_format)

datetime.datetime(2010, 1, 2, 0, 0)